In [30]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv('../2~3.데이터 수집 및 전처리/머신러닝 전처리 데이터.CSV',encoding='EUC-KR')

In [5]:
data=data.drop(['Unnamed: 0'], axis=1)

In [6]:
data

,시군구,단지명,전용면적(㎡),계약년월,계약일,계약날짜,층,건축나이,거래건수,기준금리,법정동코드,지하철역수,거래금액(만원)
0,광주광역시 광산구 도산동,대덕1,59.3100,202303,18,2023-03-18,4,33,20,3.5,2920010200,1,9800
1,광주광역시 광산구 도산동,대주피오레1차,84.9800,202302,7,2023-02-07,2,23,14,3.5,2920010200,1,21000
2,광주광역시 광산구 도산동,대주피오레1차,84.9800,202305,31,2023-05-31,5,23,16,3.5,2920010200,1,22350
3,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,9,2023-05-09,5,22,16,3.5,2920010200,1,22400
4,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,13,2023-05-13,13,22,16,3.5,2920010200,1,22000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425074,광주광역시 서구 화정동,화정우미,115.1850,200611,10,2006-11-10,17,10,131,4.5,2914011900,1,19000
425075,광주광역시 서구 화정동,화정우미,84.8300,200611,27,2006-11-27,6,10,131,4.5,2914011900,1,12000
425076,광주광역시 서구 화정동,화정우미,59.8300,200611,30,2006-11-30,2,10,131,4.5,2914011900,1,7700
425077,광주광역시 서구 화정동,화정우미,84.8400,200612,6,2006-12-06,16,10,124,4.5,2914011900,1,12170


In [8]:
pip install xgboost

                                              0.0/70.9 MB ? eta -:--:--
                                              0.1/70.9 MB 2.6 MB/s eta 0:00:27
                                              0.3/70.9 MB 4.2 MB/s eta 0:00:17
                                              0.6/70.9 MB 4.2 MB/s eta 0:00:17
                                              0.8/70.9 MB 4.5 MB/s eta 0:00:16
                                              0.9/70.9 MB 4.4 MB/s eta 0:00:16
                                              1.2/70.9 MB 4.2 MB/s eta 0:00:17
                                              1.5/70.9 MB 4.5 MB/s eta 0:00:16
                                              1.8/70.9 MB 4.5 MB/s eta 0:00:16
     -                                        2.0/70.9 MB 4.5 MB/s eta 0:00:16
     -                                        2.3/70.9 MB 4.8 MB/s eta 0:00:15
     -                                        2.4/70.9 MB 4.7 MB/s eta 0:00:15
     -                                        2.7/70.9 MB 4

In [10]:
pip install lightgbm

                                              0.0/1.3 MB ? eta -:--:--
     ----------                               0.3/1.3 MB 7.0 MB/s eta 0:00:01
     -----------------------------            1.0/1.3 MB 8.6 MB/s eta 0:00:01
     ---------------------------------------  1.3/1.3 MB 9.2 MB/s eta 0:00:01
     ---------------------------------------- 1.3/1.3 MB 7.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install tqdm

                                              0.0/77.1 kB ? eta -:--:--
     ---------------------------------------- 77.1/77.1 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [20]:
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
import lightgbm as lgb

from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import re

In [13]:
data.head(2)

,시군구,단지명,전용면적(㎡),계약년월,계약일,계약날짜,층,건축나이,거래건수,기준금리,법정동코드,지하철역수,거래금액(만원)
0,광주광역시 광산구 도산동,대덕1,59.31,202303,18,2023-03-18,4,33,20,3.5,2920010200,1,9800
1,광주광역시 광산구 도산동,대주피오레1차,84.98,202302,7,2023-02-07,2,23,14,3.5,2920010200,1,21000


In [14]:
X = data[['전용면적(㎡)', '계약년월','층', '건축나이','거래건수','기준금리','법정동코드','지하철역수']]
y = data['거래금액(만원)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
lrmodel = LinearRegression(n_jobs=-1)
ridge = Ridge(alpha=0.8, random_state=1)
lasso = Lasso(alpha = 0.01, random_state=1)
enet = ElasticNet(alpha=0.03, l1_ratio=0.01, random_state=1)
d_tree = DecisionTreeRegressor(max_depth=6, min_samples_split=10, min_samples_leaf=15, random_state=1)
rf = RandomForestRegressor(n_estimators=500,  max_depth=9, min_samples_split=50,
                           min_samples_leaf=5, random_state=1, n_jobs=-1)
model_xgb = xgb.XGBRegressor(n_estimators=500, max_depth=9, min_child_weight=5, gamma=0.1, n_jobs=-1)
model_lgb = lgb.LGBMRegressor(n_estimators=500, max_depth=9, min_child_weight=5, n_jobs=-1)

In [28]:
def RMSE(y, y_pred):
    rmse = mean_squared_error(y, y_pred) ** 0.5
    return rmse

def rmse_cv(model):
    # cv별로 학습하는 함수
    tscv = TimeSeriesSplit(n_splits=10)
    rmse_list = []
    model_name = model.__class__.__name__
    for _, (train_index, test_index) in tqdm(enumerate(tscv.split(X), start=1), desc=f'{model_name} Cross Validations...', total=10):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf = model.fit(X_train, y_train)
        pred = clf.predict(X_test)
        rmse = RMSE(y_test, pred) 
        rmse_list.append(rmse)
    return model_name, rmse_list

def print_rmse_score(model):
    # cv별 프린팅, 평균 저장
    model_name, score = rmse_cv(model)
    for i, r in enumerate(score, start=1):
    print(f'\n{model_name} mean RMSE: {np.mean(score):.4f}')
    print('='*40)
    return model_name, np.mean(score)

In [33]:
models = []
scores = []
for model in [lrmodel, ridge, lasso, enet, d_tree, rf, model_xgb, model_lgb]:
    model_name, mean_score = print_rmse_score(model)
    models.append(model_name)
    scores.append(mean_score)

LinearRegression Cross Validations...: 100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 21.54it/s]


1 FOLDS: LinearRegression RMSLE: 10084.0162
2 FOLDS: LinearRegression RMSLE: 9080.6999
3 FOLDS: LinearRegression RMSLE: 6591.1190
4 FOLDS: LinearRegression RMSLE: 5906.4291
5 FOLDS: LinearRegression RMSLE: 5854.3037
6 FOLDS: LinearRegression RMSLE: 5383.9206
7 FOLDS: LinearRegression RMSLE: 5841.3210
8 FOLDS: LinearRegression RMSLE: 8988.2098
9 FOLDS: LinearRegression RMSLE: 4756.1851
10 FOLDS: LinearRegression RMSLE: 4739.9987

LinearRegression mean RMSLE: 6722.6203


Ridge Cross Validations...: 100%|██████████████████████████████████████████████████████| 10/10 [00:00<00:00, 34.75it/s]


1 FOLDS: Ridge RMSLE: 10084.0328
2 FOLDS: Ridge RMSLE: 9080.6732
3 FOLDS: Ridge RMSLE: 6591.1187
4 FOLDS: Ridge RMSLE: 5906.4287
5 FOLDS: Ridge RMSLE: 5854.3037
6 FOLDS: Ridge RMSLE: 5383.9202
7 FOLDS: Ridge RMSLE: 5841.3209
8 FOLDS: Ridge RMSLE: 8988.2092
9 FOLDS: Ridge RMSLE: 4756.1841
10 FOLDS: Ridge RMSLE: 4739.9989

Ridge mean RMSLE: 6722.6190


Lasso Cross Validations...: 100%|██████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.98it/s]


...

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002391 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 840
[LightGBM] [Info] Number of data points in the train set: 347793, number of used features: 8
[LightGBM] [Info] Start training from score 18529.221065


LGBMRegressor Cross Validations...:  90%|██████████████████████████████████████████▎    | 9/10 [00:13<00:01,  1.84s/it]

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002563 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 876
[LightGBM] [Info] Number of data points in the train set: 386436, number of used features: 8
[LightGBM] [Info] Start training from score 17601.731094


LGBMRegressor Cross Validations...: 100%|██████████████████████████████████████████████| 10/10 [00:15<00:00,  1.57s/it]

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
1 FOLDS: LGBMRegressor RMSLE: 6138.6828
2 FOLDS: LGBMRegressor RMSLE: 4710.4239
3 FOLDS: LGBMRegressor RMSLE: 4068.0352
4 FOLDS: LGBMRegressor RMSLE: 3124.9472
5 FOLDS: LGBMRegressor RMSLE: 2360.6858
6 FOLDS: LGBMRegressor RMSLE: 2388.1207
7 FOLDS: LGBMRegressor RMSLE: 3103.2261
8 FOLDS: LGBMRegressor RMSLE: 2919.2274
9 FOLDS: LGBMRegressor RMSLE: 1539.2745
10 FOLDS: LGBMRegressor RMSLE: 1528.6945

LGBMRegressor mean RMSLE: 3188.1318


In [34]:
result_df = pd.DataFrame({'Model': models, 'Score': scores}).reset_index(drop=True)
result_df

,Model,Score
0,LinearRegression,6722.620306
1,Ridge,6722.619036
2,Lasso,6722.616620
3,ElasticNet,6715.504320
4,DecisionTreeRegressor,5866.145420
5,RandomForestRegressor,4548.431143
6,XGBRegressor,3293.185304
7,LGBMRegressor,3188.131805
